In [1]:
import numpy as np
import h5py
from skimage import io
from matplotlib import pyplot as plt
import torch
from torch.autograd import Variable
import torch.optim as optim

In [89]:
fil=h5py.File('shapemodel.h5','r')
fly=open('testface_landmarks.txt')
ftxt=fly.readlines()
for i in range(len(ftxt)):
    ftxt[i]=ftxt[i].strip('\n').split(' ')
for i in range(68):
    for j in range(2):
        ftxt[i][j]=float(ftxt[i][j])
U_label=Variable(torch.FloatTensor(np.array(ftxt).T))

d=fil['keypoints'][:][0]
s_mean=Variable(torch.FloatTensor(fil['mean_shape'][:]))   #159645*1;
s=Variable(torch.FloatTensor(fil['pca_basis'][:]))    #159645*50;
P=Variable(torch.FloatTensor(np.array([[1,0,0],[0,1,0]])))
sigma=Variable(torch.FloatTensor(fil['sigma'][:]))   #50*1;

alpha=Variable(torch.rand((50,1)),requires_grad=True)
S=s_mean+s.mm(alpha)
S=S.view(53215,3)
f=Variable(torch.rand((2,3)),requires_grad=True)
R=Variable(torch.rand((3,3)),requires_grad=True)
t=Variable(torch.rand((2)),requires_grad=True)
S_final=Variable(torch.rand(3,68))
for i in range(68):
    S_final[:,i]=S[d[i]]
U_proj=(f*P).mm(R).mm(S_final)
for i in range(68):
    U_proj[:,i]=U_proj[:,i]+t
for i in range(2):
    for j in range(68):
        U_proj[i,j]=U_proj[i,j]/torch.sum(U_proj.data[i,:])
        U_label[i,j]=U_label[i,j]/torch.sum(U_label.data[i,:])

In [90]:
loss=((alpha*alpha)/(sigma*sigma)).mean()+((U_proj-U_label)*(U_proj-U_label)).mean()

In [91]:
optimizer = optim.SGD([f,R,t,alpha], lr = 0.001)

In [92]:
loss.backward()
optimizer.step()

In [93]:
for i in range(100):
    U_label=Variable(torch.FloatTensor(np.array(ftxt).T))
    d=fil['keypoints'][:][0]
    s_mean=Variable(torch.FloatTensor(fil['mean_shape'][:]))   #159645*1;
    s=Variable(torch.FloatTensor(fil['pca_basis'][:]))    #159645*50;
    P=Variable(torch.FloatTensor(np.array([[1,0,0],[0,1,0]])))
    sigma=Variable(torch.FloatTensor(fil['sigma'][:]))   #50*1;

    alpha=alpha
    S=s_mean+s.mm(alpha)
    S=S.view(53215,3)
    f=f
    R=R
    t=t
    S_final=Variable(torch.rand(3,68))
    for i in range(68):
        S_final[:,i]=S[d[i]]
    U_proj=(f*P).mm(R).mm(S_final)
    for i in range(68):
        U_proj[:,i]=U_proj[:,i]+t
    for i in range(2):
        for j in range(68):
            U_proj[i,j]=U_proj[i,j]/torch.sum(U_proj.data[i,:])
            U_label[i,j]=U_label[i,j]/torch.sum(U_label.data[i,:])

    loss=((alpha*alpha)/(sigma*sigma)).mean()+((U_proj-U_label)*(U_proj-U_label)).mean()
    loss.backward()
    optimizer.step()

In [94]:
print(t.grad,R.grad,alpha.grad)

(Variable containing:
1.00000e-03 *
  0.0076
 -5.6213
[torch.FloatTensor of size 2]
, Variable containing:
 0.0142 -0.0548  0.1705
 0.0039 -0.0475  0.1662
 0.0000  0.0000  0.0000
[torch.FloatTensor of size 3x3]
, Variable containing:
1.00000e-08 *
  0.2414
  0.0055
 -0.0236
 -0.0250
 -0.5724
  0.4306
  0.1529
  0.0099
 -1.0464
 -0.8785
  0.8262
  0.2367
  0.2191
 -0.3227
 -0.5398
 -0.1014
 -0.4582
  0.5164
  0.3921
  0.1837
  1.0308
 -0.0133
  0.7004
 -1.3150
 -1.2425
 -0.2257
  0.5646
 -0.6952
 -0.7567
  0.4618
 -0.6105
 -0.4446
  1.3213
  1.2767
  0.4553
  1.1679
  0.6549
  1.0470
  0.8373
  0.0334
 -1.0443
  0.9279
 -0.5430
  0.2663
 -0.2967
  0.3717
  2.6511
  0.7521
 -0.1013
 -0.1247
[torch.FloatTensor of size 50x1]
)


In [99]:
print(f,R,t,alpha)

(Variable containing:
 0.1777  0.3393  0.7073
 0.7726  0.8638  0.8082
[torch.FloatTensor of size 2x3]
, Variable containing:
 0.7624  0.1711  0.5372
 0.5482  0.2312  0.8055
 0.8648  0.2920  0.2430
[torch.FloatTensor of size 3x3]
, Variable containing:
 0.9527
 0.3179
[torch.FloatTensor of size 2]
, Variable containing:
 0.4189
 0.6761
 0.3855
 0.8774
 0.3372
 0.8324
 0.8706
 0.6098
 0.0523
 0.9106
 0.0384
 0.6873
 0.0427
 0.0437
 0.1322
 0.0772
 0.7594
 0.8549
 0.5874
 0.4996
 0.8745
 0.7340
 0.8886
 0.2629
 0.6608
 0.8751
 0.2547
 0.7857
 0.1850
 0.2048
 0.4188
 0.2285
 0.6840
 0.4362
 0.1522
 0.7037
 0.9242
 0.2388
 0.8442
 0.8629
 0.5554
 0.2291
 0.8706
 0.3312
 0.2218
 0.4905
 0.6218
 0.4586
 0.8293
 0.2717
[torch.FloatTensor of size 50x1]
)


In [100]:
loss

Variable containing:
1.00000e-04 *
  1.7825
[torch.FloatTensor of size 1]

In [108]:
file=open('result.txt')

In [110]:
aa=file.readlines()